# Update (Triggers)
Updates the ETL table based on updates to EE, GEGD, and MGP tables

### Import libraries

In [1]:
# Database stack
import sqlite3

### User defined variables

In [2]:
db = "C:/gis/gaia/data/databases/gaia.db"

In [3]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TRIGGER IF EXISTS update_etl_after_ee_insert''')
conn.commit()
conn.close()

In [8]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TRIGGER update_etl_after_ee_insert
        AFTER INSERT ON ee
            BEGIN
                INSERT INTO etl (table_name, id, vendor_id, entity_id, date, geometry)
                VALUES ('EE', NEW.catalog_id, NEW.vendor_id, NEW.entity_id, NEW.acquisition_date, AsText(NEW.bounds));
            END;
''')

conn.commit()
conn.close()

In [4]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TRIGGER update_etl_after_insert
        AFTER INSERT ON ee
            BEGIN
                INSERT INTO etl (table_name, id, vendor_id, entity_id, date, geometry)
                SELECT
                   'EE' AS table_name,
                   e.catalog_id AS id,
                   e.vendor_id AS vendor_id,
                   e.entity_id AS entity_id,
                   e.acquisition_date AS date,
                   AsText(e.bounds) as geometry
            END;
            
        AFTER INSERT ON gegd
            BEGIN
                INSERT INTO etl (table_name, id, vendor_id, entity_id, date, geometry)
                SELECT
                   'GEGD' AS table_name,
                   g.legacy_id AS id,
                   NULL AS vendor_id,
                   NULL AS entity_id,
                   Date(g.acquisition_date) AS date,
                   AsText(g.geometry) AS geometry
            END;

        AFTER INSERT ON mgp
            BEGIN
                INSERT INTO etl (table_name, id, vendor_id, entity_id, date, geometry)
                SELECT
                   'mgp' AS table_name,
                   m.id AS id,
                   NULL AS vendor_id,
                   NULL AS entity_id,
                   Date(datetime) AS date,
                   AsText(m.bbox) AS geometry
            END;
''')

conn.commit()
conn.close()

OperationalError: near "END": syntax error

### Log-in to EarthExplorer
Hint: !

In [4]:
session = requests.Session()
session = security.ee_login(session)

Enter your username:   johnwallx
Enter your password:   ········


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Retreve the Area of Interest record from the `aoi` table corresponding to the supplied DAR ID

In [ ]:
gdf_aoi = functions.get_aoi(db, dar_id)
gdf_aoi

### Create the querying payload for EarthExplorer

In [ ]:
# This needs to be handled by search.GEOJSON_FOR_EE
gdf_json = json.loads(gdf_aoi['geometry'].to_json())['features'][0]['geometry']

payload = {}

data_filter = search.build_scene_filter(
    acquisition = search.build_acqusition_filter(start_date, end_date),
    spatial = search.build_spatial_filter(gdf_json),
    cloud = search.build_cloud_cover_filter()
)

max_results = 1_000

params = {"datasetName": datasetName,
          "sceneFilter": data_filter,
          "maxResults": max_results,
          "metadataType": "full",}

data = json.dumps(params)

data

### Query EarthExplorer for data

In [ ]:
url = "https://m2m.cr.usgs.gov/api/api/json/stable/scene-search"

results = session.get(url=url, data=data)
print(results.status_code)

results.json()

### Create a GeoDataFrame from query results

In [ ]:
gdf = functions.gdf_from_ee(results, dar_id)
print(gdf.shape)
gdf.head()

### Insert EarthExplorer Entity IDs into `ee` table

In [ ]:
functions.insert_pk(db, 'ee', gdf)

### Update `ee` records from EarthExplorer results

In [ ]:
columns = gdf.columns[1:]
for i, row in gdf.iterrows():
    eid = row['entity_id']
    print("Updating information for Entity ID: {}".format(eid))
    for column in columns:
        functions.database_activity(db, 'ee', eid, column, row[column])

### Select newly inserted AOIs, make a GeoDataFrame for validation

In [ ]:
gdf = functions.validate_updates(db, 'ee', gdf, dar_id)
# Note that the GDF shape matches that from the above
print(gdf.shape)
gdf.head()

# conn = sqlite3.connect(db)
# conn.enable_load_extension(True)
# conn.execute("SELECT load_extension('mod_spatialite')")

# columns_list = list(gdf.columns[:-1])
# columns_str = ', '.join(columns_list)
# sql_string = "SELECT {}, AsText(bounds) FROM ee WHERE vendor IS NOT NULL AND dar_id = {}".format(columns_str, dar_id)

# df = pd.read_sql_query(sql_string, conn)
# df = df.rename(columns={'AsText(bounds)': 'geometry'}, errors='raise')
# df['geometry'] = shapely.wkt.loads(df['geometry'])
# gdf = gpd.GeoDataFrame(df, geometry='geometry')

# conn.commit()
# conn.close()

# # Note that the GDF shape matches that from the above
# print(gdf.shape)
# gdf.head()

### Plot Areas of Interest on an Interactive Map

In [ ]:
mp = functions.quick_map(gdf, gdf_json)
mp

# End